In [4]:
import numpy as np
import torch
import torchvision
from matplotlib import pyplot as plt
from matplotlib import cm
from sparse_model import SparseLayer

from train_sparse_model import load_mnist_data
from train_sparse_model import plot_filters

In [5]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
if device == "cpu":
    batch_size = 64
else:
    batch_size = 4096

train_loader = load_mnist_data(batch_size)
example_data, example_targets = next(iter(train_loader))

idx = 0
num_img = 32
num_filters = 784
imgs = example_data[idx:idx+num_img, 0, :, :].to(device)
sparse_layer = SparseLayer(imgs.shape[1], imgs.shape[2], num_filters)
sparse_layer.to(device)

learning_rate = 1e-3
filter_optimizer = torch.optim.Adam(sparse_layer.parameters(),
                                    lr=learning_rate)

/home/cm3786@drexel.edu/.pyenv/versions/3.9.6/envs/jupyter3.9.6/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
# for _ in range(20):
#     activations = sparse_layer(imgs)
#     loss = sparse_layer.loss(imgs, activations)
for epoch in range(1):
    for local_batch, local_labels in train_loader:
        local_batch = local_batch.to(device)
        local_labels = local_labels.to(device)
        activations = sparse_layer(local_batch[:, 0, :, :])
        loss = sparse_layer.loss(local_batch[:, 0, :, :], activations)
        print('loss={}'.format(loss))

        filter_optimizer.zero_grad()
        loss.backward()
        filter_optimizer.step()
        sparse_layer.normalize_weights()

loss=339.3970031738281
loss=302.4366455078125
loss=265.2920837402344
loss=246.69708251953125
loss=232.9738311767578
loss=222.8897705078125
loss=215.71499633789062
loss=207.3680877685547
loss=201.38571166992188
loss=195.90518188476562
loss=190.6444549560547
loss=185.08001708984375
loss=182.55438232421875
loss=178.92910766601562
loss=175.71067810058594


In [ ]:
activations = sparse_layer(imgs)
reconstructions = sparse_layer.reconstructions(
    activations).cpu().detach().numpy()

print("SHAPES")
print(imgs.shape)
print(reconstructions.shape)

fig = plt.figure()

img_to_show = 3
for i in range(img_to_show):
    # original
    plt.subplot(img_to_show, 2, i*2 + 1)
    plt.tight_layout()
    plt.imshow(example_data[idx+i, 0, :, :], cmap='gray',
               interpolation='none')
    plt.title("Original Image\nGround Truth: {}".format(
        example_targets[idx]))
    plt.xticks([])
    plt.yticks([])

    # reconstruction
    plt.subplot(img_to_show, 2, i*2 + 2)
    plt.tight_layout()
    plt.imshow(reconstructions[i, :, :], cmap='gray',
               interpolation='none')
    plt.title("Reconstruction")
    plt.xticks([])
    plt.yticks([])